<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/latest_with_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!npm install -g localtunnel
!pip install flask flask-ngrok tensorflow joblib pykalman scipy
!pip install PyWavelets

from flask import Flask, jsonify
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from scipy.signal import butter, filtfilt
from pykalman import KalmanFilter
import subprocess

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧Requirement already satisfied: flask in /usr/local/lib/python3.11/dist-packages (3.1.0)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.7 MB/s eta 0:00:00


In [ ]:
app = Flask(__name__)

# Setup for your EEG signal processing
sampling_rate_original = 2500
sampling_rate_final = 200
freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

def bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

def downsample(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]

def generate_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg = []
    for _ in range(62):
        alpha = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, 3, t.shape)
        signal = delta + theta + alpha + beta + gamma + noise
        eeg.append(np.clip(signal, -70, 70))
    return np.array(eeg)

def extract_de_lds():
    eeg_data = generate_eeg()
    de_feats = []
    for ch in eeg_data:
        ch = downsample(ch, sampling_rate_original, sampling_rate_final)
        band_feats = []
        for _, (low, high) in freq_bands.items():
            filtered = bandpass_filter(ch, low, high, sampling_rate_final)
            var = np.var(filtered)
            de = 0.5 * np.log(2 * np.pi * np.e * var + 1e-8)
            band_feats.append(de)
        smoothed, _ = kf.filter(np.array(band_feats).reshape(-1, 1))
        de_feats.append(smoothed.flatten())
    de_feats = np.array(de_feats)
    de_min, de_max = np.min(de_feats), np.max(de_feats)
    scaled = 15 + (de_feats - de_min) / (de_max - de_min + 1e-8) * (27 - 15)
    return scaled.flatten()

@app.route('/predict', methods=['GET'])
def predict():
    sample = extract_de_lds().reshape((1, 1, 310))
    model = load_model('/content/eeg_emotion_gan_model.h5')
    encoder = joblib.load('/content/eeg_label_encoder.pkl')
    prediction = model.predict(sample)

    # print(f"Model Prediction: {prediction}")

    label = encoder.inverse_transform(np.argmax(prediction, axis=1))[0]
    print(f"Predicted Emotion: {label}")
    return jsonify({'emotion': label})

# Run LocalTunnel to expose the Flask API with a custom subdomain for static URL
def run_localtunnel():
    subdomain = "eegemotion"  # Replace with your desired subdomain
    process = subprocess.Popen(['lt', '--port', '5000', '-s', subdomain], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode('utf-8').strip())  # Show the URL output from LocalTunnel

# Start the LocalTunnel process and Flask app
if __name__ == "__main__":
    from threading import Thread
    # Run LocalTunnel in a separate thread
    thread = Thread(target=run_localtunnel)
    thread.start()
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://eegemotion.loca.lt


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step


INFO:werkzeug:127.0.0.1 - - [11/May/2025 16:21:42] "GET /predict HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [11/May/2025 16:34:48] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Neutral


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step


INFO:werkzeug:127.0.0.1 - - [11/May/2025 16:34:54] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Fear


In [ ]:
app = Flask(__name__)

# Setup for your EEG signal processing
sampling_rate_original = 2500
sampling_rate_final = 200
freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

def bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

def downsample(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]

def generate_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg = []
    for _ in range(62):
        alpha = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, 3, t.shape)
        signal = delta + theta + alpha + beta + gamma + noise
        eeg.append(np.clip(signal, -70, 70))
    return np.array(eeg)

def extract_de_lds(eeg_data):
    de_feats = []
    for ch in eeg_data:
        ch = downsample(ch, sampling_rate_original, sampling_rate_final)
            eeg_data = [bandpass_filter(ch, 1, 50, sampling_rate_final) for ch in eeg_data]

        band_feats = []
        for _, (low, high) in freq_bands.items():
            filtered = bandpass_filter(ch, low, high, sampling_rate_final)
            var = np.var(filtered)
            de = 0.5 * np.log(2 * np.pi * np.e * var + 1e-8)
            band_feats.append(de)
        smoothed, _ = kf.filter(np.array(band_feats).reshape(-1, 1))
        de_feats.append(smoothed.flatten())
    de_feats = np.array(de_feats)
    de_min, de_max = np.min(de_feats), np.max(de_feats)
    scaled = 15 + (de_feats - de_min) / (de_max - de_min + 1e-8) * (27 - 15)
    return scaled.flatten()

@app.route('/predict', methods=['GET'])
def predict():
    eeg_data = generate_eeg()  # Generate EEG data for emotion prediction
    sample = extract_de_lds(eeg_data).reshape((1, 1, 310))

    model = load_model('/content/eeg_emotion_gan_model.h5')
    encoder = joblib.load('/content/eeg_label_encoder.pkl')
    prediction = model.predict(sample)

    label = encoder.inverse_transform(np.argmax(prediction, axis=1))[0]
    print(f"Predicted Emotion: {label}")

    # Send both emotion prediction and EEG data to front-end
    return jsonify({
        'emotion': label,
        'eeg_data': eeg_data.tolist()  # Send the raw EEG data to display as a waveform
    })

# Run LocalTunnel to expose the Flask API with a custom subdomain for static URL
def run_localtunnel():
    subdomain = "eegemotion"  # Replace with your desired subdomain
    process = subprocess.Popen(['lt', '--port', '5000', '-s', subdomain], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode('utf-8').strip())  # Show the URL output from LocalTunnel

# Start the LocalTunnel process and Flask app
if __name__ == "__main__":
    from threading import Thread
    # Run LocalTunnel in a separate thread
    thread = Thread(target=run_localtunnel)
    thread.start()
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://eegemotion.loca.lt


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step


INFO:werkzeug:127.0.0.1 - - [11/May/2025 20:37:18] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Fear


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step


INFO:werkzeug:127.0.0.1 - - [11/May/2025 20:50:07] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Sad


In [ ]:
#with mspca and welch

from flask import Flask, jsonify
import numpy as np
from scipy.signal import butter, filtfilt, welch
from pykalman import KalmanFilter
from tensorflow.keras.models import load_model
import joblib
import pywt
from sklearn.decomposition import PCA
import subprocess
from threading import Thread

app = Flask(__name__)

# --- EEG Config ---
sampling_rate_original = 2500
sampling_rate_final = 200
segment_length = sampling_rate_final * 1

freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}

# --- Kalman Filter Setup ---
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

# --- Filtering & Downsampling ---
def bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

def downsample(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]

# --- EEG Generator ---
def generate_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg = []
    for _ in range(62):
        alpha = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, 3, t.shape)
        signal = delta + theta + alpha + beta + gamma + noise
        eeg.append(np.clip(signal, -70, 70))
    return np.array(eeg)

# --- MSPCA Denoising ---
def mspca_denoise(eeg_data, wavelet='db4', level=4):
    denoised_eeg = []
    for channel in eeg_data:
        coeffs = pywt.wavedec(channel, wavelet, level=level)
        denoised_coeffs = []
        for c in coeffs:
            if len(c) > 1:
                c = c.reshape(-1, 1)
                pca = PCA(n_components=1)
                transformed = pca.fit_transform(c)
                reconstructed = pca.inverse_transform(transformed).flatten()
                denoised_coeffs.append(reconstructed)
            else:
                denoised_coeffs.append(c)
        denoised_signal = pywt.waverec(denoised_coeffs, wavelet)
        denoised_eeg.append(denoised_signal[:len(channel)])
    return np.array(denoised_eeg)

# --- DE_LDS with Welch & MSPCA ---
def extract_de_lds(eeg_data):
    eeg_data = [downsample(ch, sampling_rate_original, sampling_rate_final) for ch in eeg_data]
    eeg_data = [bandpass_filter(ch, 1, 50, sampling_rate_final) for ch in eeg_data]
    eeg_data = mspca_denoise(np.array(eeg_data))

    de_features = []
    for ch_data in eeg_data:
        freqs, psd = welch(ch_data, fs=sampling_rate_final, nperseg=min(200, segment_length))
        band_de = []
        for _, (low, high) in freq_bands.items():
            band_power = psd[(freqs >= low) & (freqs <= high)]
            band_de_value = 0.5 * np.log(2 * np.pi * np.e * (np.mean(band_power) + 1e-6))
            band_de.append(band_de_value)
        smoothed_de, _ = kf.filter(np.array(band_de).reshape(-1, 1))
        de_features.append(smoothed_de.flatten())
    de_features = np.array(de_features)

    # Normalize
    de_min, de_max = np.min(de_features), np.max(de_features)
    scaled = 15 + (de_features - de_min) / (de_max - de_min + 1e-8) * (27 - 15)
    return scaled.flatten()

# --- API Route ---
@app.route('/predict', methods=['GET'])
def predict():
    eeg_data = generate_eeg()
    sample = extract_de_lds(eeg_data).reshape((1, 1, 310))

    model = load_model('/content/eeg_emotion_gan_model.h5')
    encoder = joblib.load('/content/eeg_label_encoder.pkl')
    prediction = model.predict(sample)
    label = encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    print(f"Predicted Emotion: {label}")
    return jsonify({'emotion': label, 'eeg_data': eeg_data.tolist()})

# --- LocalTunnel Setup ---
def run_localtunnel():
    subdomain = "eegemotion"
    process = subprocess.Popen(['lt', '--port', '5000', '-s', subdomain], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        print(line.decode('utf-8').strip())

# --- Run App ---
if __name__ == "__main__":
    from threading import Thread
    # Run LocalTunnel in a separate thread
    thread = Thread(target=run_localtunnel)
    thread.start()
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://eegemotion.loca.lt


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step


INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:41:43] "GET /predict HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [12/May/2025 16:03:02] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Anger


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step


INFO:werkzeug:127.0.0.1 - - [12/May/2025 16:04:19] "GET /predict HTTP/1.1" 200 -


Predicted Emotion: Anger
